In [1]:
import numpy as np
import os
from PIL import Image

from scipy.ndimage import shift
from skimage import io

import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interactive


In [2]:
image1 = os.path.join("data",  "siemens_star.tif")
assert os.path.exists(image1), f"File does not exist: {image1}"


In [3]:
# config = {'chip1': {'xoffset': 1,
#                     'yoffset': 1,
#                     'description': 'top right chip'},
#             'chip2': {'xoffset': 0,
#                     'yoffset': 0,
#                     'description': 'top left and reference chip'},
#             'chip3': {'xoffset': 0,
#                         'yoffset': 1,
#                         'description': 'bottom left chip'},
#             'chip4': {'xoffset': 1,
#                     'yoffset': 2,
#                     'description': 'bottom right chip'}
# }

shift_config = {'chip1': {'xoffset': 2.4,
                    'yoffset': 1,
                    'description': 'top right chip'},
            'chip2': {'xoffset': 0,
                    'yoffset': 0,
                    'description': 'top left and reference chip'},
            'chip3': {'xoffset': 0,
                        'yoffset': 1,
                        'description': 'bottom left chip'},
            'chip4': {'xoffset': 2.3,
                    'yoffset': 2.3,
                    'description': 'bottom right chip'}
}



In [4]:
# Load images
data_images1 = np.array(Image.open(image1))

In [5]:
# # preview
# fig, ax = plt.subplots(figsize=(10, 10))
# ax.imshow(data_images1)
# ax.set_title("Image 1: normalized_sample_8997_obs_8999")
# # ax.axis('off')
# plt.show()

# Applying shift correction based on the configuration

In [6]:
region = data_images1[0:256, 256:]
region[np.isnan(region)] = 0
region[np.isinf(region)] = 0

chips1_shift = (shift_config['chip1']['yoffset'], shift_config['chip1']['xoffset'])
shifted_data = shift(region, shift=chips1_shift, order=3)
new_data_images = data_images1.copy()
new_data_images[0:256, 256:] = 0
new_data_images[0:256, 256:] = shifted_data

region2 = new_data_images[256:, 0:256]
region2[np.isnan(region2)] = 0
region2[np.isinf(region2)] = 0
chips3_shift = (shift_config['chip3']['yoffset'], shift_config['chip3']['xoffset'])
shifted_data2 = shift(region2, shift=chips3_shift, order=3)
new_data_images[256:, 0:256] = 0
new_data_images[256:, 0:256] = shifted_data2

region3 = new_data_images[256:, 256:]
region3[np.isnan(region3)] = 0
region3[np.isinf(region3)] = 0
chips4_shift = (shift_config['chip4']['yoffset'], shift_config['chip4']['xoffset'])
shifted_data3 = shift(region3, shift=chips4_shift, order=3)
new_data_images[256:, 256:] = 0
new_data_images[256:, 256:] = shifted_data3

# # preview
# fig, axs = plt.subplots(ncols=1, figsize=(10, 10))
# axs.imshow(new_data_images)
# axs.set_title("new data images")
# # axs.axis('off')

In [7]:

_data_images = new_data_images.copy()

def plot_circles(radius_coeff=1.0, center_x=271, center_y=258):

    fig, axs = plt.subplots(ncols=1, figsize=(8, 8))
    axs.imshow(_data_images)
    ny, nx = _data_images.shape
    
    axs.set_title("new data images")
    # axs.axis('off')

    # let's plot some circles to see how well they align
    theta = np.linspace(0, 2*np.pi, 100)
    radii = np.array([1, 20, 40, 60, 80, 100])*radius_coeff
    center = (center_y, center_x)
    for r in radii:
        x = center[1] + r * np.cos(theta)
        y = center[0] + r * np.sin(theta)
        axs.plot(x, y, 'r--', linewidth=1)
    
    
    
    plt.show()

interactive_plot = interactive(plot_circles,
                               radius_coeff=widgets.FloatSlider(value=1.82, min=0.5, max=3.0, step=0.01, description='Radius Coeff:'),
                               center_x=widgets.IntSlider(value=271, min=200, max=310, step=0.1, description='Center X:'),
                               center_y=widgets.IntSlider(value=259, min=200, max=310, step=0.1, description='Center Y:'),
                              
                               
                              )
display(interactive_plot)

interactive(children=(FloatSlider(value=1.82, description='Radius Coeff:', max=3.0, min=0.5, step=0.01), IntSl…